In [1]:
import re
import pandas as pd
import numpy as np
import funcs 
import importlib
import unicodecsv as csv
import pickle
import random
import gensim
from gensim import corpora, models
from timeit import default_timer as timer
from datetime import datetime, timedelta
import nltk
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
import spacy
from collections import Counter
import pyLDAvis
import pyLDAvis.gensim
from sklearn.model_selection import train_test_split

importlib.reload(funcs)

<module 'funcs' from 'D:\\_projekty\\praca\\funcs.py'>

## Załadowanie danych oczyszczonych i zlematyzowanych oraz dalsze czyszczenie

In [2]:
dane = pd.read_csv('00_dane/dane_TM.csv',  sep='|', encoding='UTF-8')

In [3]:
%%time
dane['speech_rawest'] = dane['speech_rawest'].apply(lambda x: re.sub('[\W\d]',' ', x.lower()))
nlp = spacy.load('pl_model')
stop = set(nlp.Defaults.stop_words)
stop.add('zł')
stop.add('pkt')
stop.add('art')
stop.add('ustawy')
stop.add('r')
dane['speech_rawest'] = dane['speech_rawest'].apply(lambda x: [word for word in x.split() if word not in stop])
#czas dla apply : 19s

<unknown>:1: DeprecationWarning: invalid escape sequence \W


Wall time: 44.9 s


## Analiza za: https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24)

Zmiana - dodanie bigramów do korpusu

In [4]:
corp = dane['speech_rawest']
bigram = gensim.models.Phrases(corp, min_count=20, threshold=50) 
bigram_mod = gensim.models.phrases.Phraser(bigram)
corp = [bigram_mod[doc] for doc in corp]

In [29]:
corp[100]

['chodzić',
 'rzecz',
 'ściśle',
 'techniczny',
 'klub',
 'podzielać',
 'mnieman',
 'rząd',
 'zmiana',
 'mówać',
 'projekt',
 'ustawa',
 'winnać',
 'wprowadzić',
 'sprawa',
 'chodzić',
 'zmiana',
 'dotyczyć',
 'ogół',
 'wyłącznie',
 'osoba',
 'podeszły',
 'wiek',
 'sytuacja',
 'dotychczas',
 'przekazanie',
 'sprawa',
 'sąd',
 'wojewódzki',
 'warszawa',
 'narażać',
 'osoba',
 'konieczność',
 'dojazd',
 'stolica',
 'dodlegiegszy',
 'część',
 'kraj',
 'przekazanie',
 'sprawa',
 'właściwość',
 'sąd',
 'wojewódzki',
 'zatem',
 'akt',
 'prosty',
 'uzasadnić',
 'okoliczność',
 'sytuacja',
 'zmiana',
 'ustawa',
 'uznanie',
 'nieważny',
 'orzeczenie',
 'wydać',
 'wobec',
 'osoba',
 'represjonować',
 'działalność',
 'rzecz',
 'niepodległy',
 'byt',
 'państwo',
 'polski',
 'zupełnie',
 'oczywisty',
 'stan',
 'rzecz',
 'imię',
 'klub',
 'przyłączać',
 'opinia',
 'autor',
 'projekt',
 'podzielać',
 'słuszność',
 'argumentacja',
 'zawarty',
 'uzasadnienie']

In [31]:
bigram[corp[100]]

['chodzić',
 'rzecz',
 'ściśle',
 'techniczny',
 'klub',
 'podzielać',
 'mnieman',
 'rząd',
 'zmiana',
 'mówać',
 'projekt',
 'ustawa',
 'winnać',
 'wprowadzić',
 'sprawa',
 'chodzić',
 'zmiana',
 'dotyczyć',
 'ogół',
 'wyłącznie',
 'osoba',
 'podeszły_wiek',
 'sytuacja',
 'dotychczas',
 'przekazanie',
 'sprawa',
 'sąd',
 'wojewódzki',
 'warszawa',
 'narażać',
 'osoba',
 'konieczność',
 'dojazd',
 'stolica',
 'dodlegiegszy',
 'część',
 'kraj',
 'przekazanie',
 'sprawa',
 'właściwość',
 'sąd',
 'wojewódzki',
 'zatem',
 'akt',
 'prosty',
 'uzasadnić',
 'okoliczność',
 'sytuacja',
 'zmiana',
 'ustawa',
 'uznanie_nieważny',
 'orzeczenie',
 'wydać',
 'wobec',
 'osoba',
 'represjonować',
 'działalność',
 'rzecz',
 'niepodległy_byt',
 'państwo',
 'polski',
 'zupełnie',
 'oczywisty',
 'stan',
 'rzecz',
 'imię',
 'klub',
 'przyłączać',
 'opinia',
 'autor',
 'projekt',
 'podzielać',
 'słuszność',
 'argumentacja',
 'zawarty',
 'uzasadnienie']

In [34]:
corp[100]

['chodzić',
 'rzecz',
 'ściśle',
 'techniczny',
 'klub',
 'podzielać',
 'mnieman',
 'rząd',
 'zmiana',
 'mówać',
 'projekt',
 'ustawa',
 'winnać',
 'wprowadzić',
 'sprawa',
 'chodzić',
 'zmiana',
 'dotyczyć',
 'ogół',
 'wyłącznie',
 'osoba',
 'podeszły_wiek',
 'sytuacja',
 'dotychczas',
 'przekazanie',
 'sprawa',
 'sąd',
 'wojewódzki',
 'warszawa',
 'narażać',
 'osoba',
 'konieczność',
 'dojazd',
 'stolica',
 'dodlegiegszy',
 'część',
 'kraj',
 'przekazanie',
 'sprawa',
 'właściwość',
 'sąd',
 'wojewódzki',
 'zatem',
 'akt',
 'prosty',
 'uzasadnić',
 'okoliczność',
 'sytuacja',
 'zmiana',
 'ustawa',
 'uznanie_nieważny',
 'orzeczenie',
 'wydać',
 'wobec',
 'osoba',
 'represjonować',
 'działalność',
 'rzecz',
 'niepodległy_byt',
 'państwo',
 'polski',
 'zupełnie',
 'oczywisty',
 'stan',
 'rzecz',
 'imię',
 'klub',
 'przyłączać',
 'opinia',
 'autor',
 'projekt',
 'podzielać',
 'słuszność',
 'argumentacja',
 'zawarty',
 'uzasadnienie']

In [52]:
dictionary = gensim.corpora.Dictionary(corp)

Usuwanie słów rzadkich i zbyt częstych i ograniczeni liczby tokenów

In [54]:
dictionary.filter_extremes(no_above=0.5, no_below=5 )

In [55]:
len(dictionary)

62004

## Bag of Words
Dla każdego dokumentu tworzymy słownik zawierający informację jak dużo słów się pojawia i jak często.

In [44]:
BOW = [dictionary.doc2bow(speech) for speech in corp]

## TF-IDF
Oblicznie wartości tf-idf

In [46]:
tfidf = models.TfidfModel(BOW)
corpus_tfidf = tfidf[BOW]


## LDA oparte o BOW

In [47]:
start = timer()
lda_model = gensim.models.LdaMulticore(BOW, id2word=dictionary,num_topics=100, passes=1, workers=2)
end = timer()
print(timedelta(seconds=timer()-start))


0:04:32.140649


In [51]:
funcs.print_topics(lda_model.print_topics(20))

,0,1,2,3,4,5,6,7,8,9
Topic 37,osoba,zapis,uprawnienie,policja,przypadek,posiadać,obywatel,państwo,sprawa,dotyczyć
Topic 56,samochód,rybołówstwo,stadion,rybak,polski,hut,rybacki,częstochowo,rząd,połowa
Topic 89,praca,pracownik,pracodawca,zawodowy,czas,prawo,państwowy_inspekcja,zatrudnienie,kodeks,kobieta
Topic 83,szkoła,nauczyciel,edukacja,szkół,dziecko,oświata,minister,młodzież,uczeń,system
Topic 48,zakład,zdrowotny,zakład_opieka,zdrowie,służba,pracownik,publiczny,minister,jednostka,powiatowy
Topic 71,wzrost,budżetowy,budżet,poziom,płaca,wydatek,wynagrodzić,wskaźnik,wysoki,wynagrodzenie
Topic 67,bank,bankowy,kredyt,finansowy,kredytowy,udzielać,klient,sytuacja,informacja,amber_gold
Topic 50,komisja,projekt,wniosek,poseł,sejm,izba,zostać,czytać,uchwała,wniosek_mniejszość
Topic 61,rolnik,rolnictwo,rolny,gospodarstwo,rolniczy,polski,rynek,produkcja,wieś,unia_europejski
Topic 30,postępowanie,kodeks,projekt,cywilny,zmiana,przepis,prawo,komornik,sądowy,sprawa


In [58]:
w = "państwo"
print("Słowo {} - {} pojawia się w korpusie {} razy i występuje w {} dokumentach ({:04.1f}%).".format(dictionary.token2id[w],
                                                                                         w,
                                                                                         dictionary.cfs[dictionary.token2id[w]],
                                                                                         dictionary.dfs[dictionary.token2id[w]],
                                                                                         100*dictionary.dfs[dictionary.token2id[w]]/158879))

Słowo 107 - państwo pojawia się w korpusie 261503 razy i występuje w 78920 dokumentach (49.7%).


### Modyfikacje parametrów słownika

In [59]:
dictionary.filter_extremes(no_above=0.3, no_below=5 )
print(len(dictionary))
BOW = [dictionary.doc2bow(speech) for speech in corp]
tfidf = models.TfidfModel(BOW)
corpus_tfidf = tfidf[BOW]
start = timer()
lda_model = gensim.models.LdaMulticore(BOW, id2word=dictionary,num_topics=100, passes=1, workers=2)
end = timer()
print(timedelta(seconds=timer()-start))

61978
0:04:12.926873


In [63]:
funcs.print_topics(lda_model.print_topics(10))

,0,1,2,3,4,5,6,7,8,9
Topic 96,nauczyciel,wynagrodzić,oświata,płaca,system,wynagrodzenie,zawodowy,edukacja,pracownik,karta_nauczyciel
Topic 85,konkordat,ratyfikacja_konkordat,spółka_węglowy,dolar,informacja_kryminalny,konsolidacja,naftowy,rafineria,rosyjski,gaz
Topic 0,sejm,wniosek,izba,konstytucja,pewien,klub,problem,uchwała,kwestia,marszałek
Topic 74,rynek,produkcja,producent,towar,produkt,import,konsument,eksport,celny,produkować
Topic 43,pomoc,osoba,świadczenie,społeczny,udzielać,socjalny,uprawnić,kryterium,przyznawać,dom
Topic 92,szkoła,dziecko,edukacja,szkół,uczeń,rodzic,młodzież,nauczyciel,szkolny,reforma
Topic 67,środek,fundusz,program,rozwój,finansowy,realizacja,wykorzystać,pieniądz,ramy,przeznaczyć
Topic 34,konwencja,międzynarodowy,kościół,protokół,ratyfikacja,umowa,rzeczypospolity,organizacja,ratyfikować,dzień
Topic 94,unia_europejski,europejski,członkowski,kraj,parlament,unia,unijny,wspólnota,traktat,europa
Topic 31,gospodarczy,społeczny,gospodarka,polityka,ekonomiczny,rozwój,problem,rozwiązanie,nowy,proces


# Optymalziacja parametrów

Wydzielenie próby do testów.

In [70]:
test1, _ = train_test_split(corp,train_size=0.3,random_state=666)  
len(test1)

47663

In [76]:
start = timer()

no_a = [0.1, 0.2, 0.3, 0.4]
num_t = [20, 30, 40, 50]
p = [3, 5, 7, 10]
res = pd.DataFrame()

for n_a in no_a:
    for n_t in num_t:
        for pas in p:
            dictionary = gensim.corpora.Dictionary(test1)
            dictionary.filter_extremes(no_above=n_a, no_below=50)
            BOW = [dictionary.doc2bow(speech) for speech in test1]
            tfidf = models.TfidfModel(BOW)
            corpus_tfidf = tfidf[BOW]

            lda_model = gensim.models.LdaMulticore(BOW, id2word=dictionary,num_topics=n_t, passes=pas, workers=2)
            coherence_model_lda = gensim.models.CoherenceModel(model=lda_model, texts=test1, dictionary=dictionary, coherence='c_v')
            coherence_lda = coherence_model_lda.get_coherence()
            print(n_a, n_t, pas, coherence_lda)
            res = res.append([[n_a, n_t, pas, coherence_lda]], ignore_index=True)
end = timer()
print(timedelta(seconds=timer()-start))        

0.1 20 3 0.5749356486929605
0.1 20 5 0.6134281662737379
0.1 20 7 0.6131149632323716
0.1 20 10 0.6263404570587572
0.1 30 3 0.5842868828078583
0.1 30 5 0.6069223805253418
0.1 30 7 0.6247842150899946
0.1 30 10 0.604681418451397
0.1 40 3 0.5935077046343649
0.1 40 5 0.5964434484771048
0.1 40 7 0.6167442032295201
0.1 40 10 0.6235714372941267
0.1 50 3 0.5835001216967965
0.1 50 5 0.60378793873292
0.1 50 7 0.6203304304093984
0.1 50 10 0.6094241618233623
0.2 20 3 0.5115435864453216
0.2 20 5 0.553152549655796
0.2 20 7 0.559388193882679
0.2 20 10 0.5712185461418386
0.2 30 3 0.5420336166413892
0.2 30 5 0.5565809076020201
0.2 30 7 0.5770810313152628
0.2 30 10 0.5861793661566017
0.2 40 3 0.5673751963419212
0.2 40 5 0.5806919634622398
0.2 40 7 0.5824970182062926
0.2 40 10 0.5897691926549721
0.2 50 3 0.5483208955911318
0.2 50 5 0.5819417102842771
0.2 50 7 0.5771241542309384
0.2 50 10 0.5926767146896059
0.3 20 3 0.5201709457387972
0.3 20 5 0.5284423572099965
0.3 20 7 0.5286668027650774
0.3 20 10 0.55619

### Dwa modele do porównania

In [9]:
dictionary = gensim.corpora.Dictionary(corp)
dictionary.filter_extremes(no_above=0.1, no_below=50)
BOW = [dictionary.doc2bow(speech) for speech in corp]
tfidf = models.TfidfModel(BOW)
corpus_tfidf = tfidf[BOW]

lda_model_1 = gensim.models.LdaMulticore(BOW, id2word=dictionary,num_topics=40, passes=10, workers=2)
coherence_model_lda = gensim.models.CoherenceModel(model=lda_model_1, texts=corp, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print("0.1, 40, 10, ", coherence_lda)

dictionary2 = gensim.corpora.Dictionary(corp)
dictionary2.filter_extremes(no_above=0.2, no_below=50)
BOW = [dictionary2.doc2bow(speech) for speech in corp]
tfidf = models.TfidfModel(BOW)
corpus_tfidf = tfidf[BOW]

lda_model_2 = gensim.models.LdaMulticore(BOW, id2word=dictionary2,num_topics=40, passes=10, workers=2)
coherence_model_lda = gensim.models.CoherenceModel(model=lda_model_2, texts=corp, dictionary=dictionary2, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print("0.2, 40, 10, ", coherence_lda)

0.1, 40, 10,  0.6281662586115726
0.2, 40, 10,  0.5973166449925559


In [11]:
a = funcs.print_topics(lda_model_1.print_topics(-1))

In [12]:
print(a)

                         0                      1                     2  \
Topic 0        zagraniczny         międzynarodowy                 umowa   
Topic 1              pytać                  robić                 złoty   
Topic 2                pis              głos_sala  platform_obywatelski   
Topic 3            agencja           nieruchomość                 grunt   
Topic 4              świat                kultura                 naród   
Topic 5      ubezpieczenie            świadczenie            pracodawca   
Topic 6               dana               dokument                   typ   
Topic 7            zdrowie                 lekarz               pacjent   
Topic 8       spółdzielnia           spółdzielczy     bank_spółdzielczy   
Topic 9               kara                  karny          przestępstwo   
Topic 10        inwestycja                    las                 teren   
Topic 11     administracja            województwo            regionalny   
Topic 12           reform

In [13]:
b = funcs.print_topics(lda_model_2.print_topics(-1))

In [14]:
print(b)

                        0                1                      2  \
Topic 0       konstytucja        prezydent                 wybory   
Topic 1         telewizja          krajowy                 medium   
Topic 2           dziecko          rodzina               rzecznik   
Topic 3        mieszkanie         wojskowy           mieszkaniowy   
Topic 4             droga       inwestycja         infrastruktura   
Topic 5          własność     nieruchomość                agencja   
Topic 6            budżet        budżetowy                wydatek   
Topic 7          pieniądz         naprawdę                 prosty   
Topic 8           zdrowie           lekarz                pacjent   
Topic 9            spółka     prywatyzacja                  skarb   
Topic 10           system          wydawać                  różny   
Topic 11         kontrola     sprawozdanie                 prezes   
Topic 12          program           rozwój            gospodarczy   
Topic 13  unia_europejski       eu

In [15]:
pickle.dump(lda_model_1, open('lda_01.p', 'wb'))
pickle.dump(lda_model_2, open('lda_02.p', 'wb'))

In [5]:
dictionary3 = gensim.corpora.Dictionary(corp)
dictionary3.filter_extremes(no_above=0.3, no_below=50)
BOW = [dictionary3.doc2bow(speech) for speech in corp]
tfidf = models.TfidfModel(BOW)
corpus_tfidf = tfidf[BOW]

In [6]:
lda_model_3 = gensim.models.LdaMulticore(BOW, id2word=dictionary3,num_topics=40, passes=10, workers=2)

In [7]:
pickle.dump(lda_model_3, open('lda_03.p', 'wb'))

In [8]:
coherence_model_lda = gensim.models.CoherenceModel(model=lda_model_3, texts=corp, dictionary=dictionary3, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print("0.3, 40, 10, ", coherence_lda)

0.3, 40, 10,  0.5796464287306657


In [9]:
c = funcs.print_topics(lda_model_3.print_topics(-1))

In [10]:
print(c)

                        0              1                      2  \
Topic 0           dziecko        rodzina                  osoba   
Topic 1       gospodarczy        problem              działanie   
Topic 2       konstytucja  konstytucyjny              prezydent   
Topic 3          poprawka          senat                   klub   
Topic 4              kraj       polityka            zagraniczny   
Topic 5   unia_europejski     europejski                  umowa   
Topic 6             nauka        naukowy               uczelnia   
Topic 7            spółka   prywatyzacja                  skarb   
Topic 8              bank        bankowy                  rynek   
Topic 9             rynek      produkcja              producent   
Topic 10         kontrola           izba           sprawozdanie   
Topic 11           środek        program                fundusz   
Topic 12             kraj          celny                  towar   
Topic 13         wojskowy         służba        obrona_narodow

In [14]:
pyLDAvis.enable_notebook()

dictionary2 = gensim.corpora.Dictionary(corp)
dictionary2.filter_extremes(no_above=0.2, no_below=50)
BOW = [dictionary3.doc2bow(speech) for speech in corp]
lda_model_2 =pickle.load(open("lda_02.p", "rb"))

In [ ]:
vis = pyLDAvis.gensim.prepare(lda_model_2, BOW, dictionary2)
vis

In [32]:
test1, _ = train_test_split(dane['speech_rawest'],test_size=0.5)   

In [ ]:
%%time
coherence_model_lda = gensim.models.CoherenceModel(model=lda_model, texts=test1, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)
#Coherence Score:  0.5810685000684818

In [79]:
start = timer()
dictionary = gensim.corpora.Dictionary(dane['speech_rawest'])
dictionary.filter_extremes(no_above=0.3, no_below=50 )
print(len(dictionary))
BOW = [dictionary.doc2bow(speech) for speech in dane['speech_rawest']]
tfidf = models.TfidfModel(BOW)
corpus_tfidf = tfidf[BOW]

lda_model = gensim.models.LdaMulticore(BOW, id2word=dictionary,num_topics=35, passes=1, workers=2)
end = timer()
print(timedelta(seconds=timer()-start))
funcs.print_topics(lda_model.print_topics(-1))

18200
0:03:24.034137


,0,1,2,3,4,5,6,7,8,9
Topic 0,środowisko,ochrona,odpad,woda,wodny,gospodarka,ekologiczny,naturalny,zasób,opłata
Topic 1,pracownik,pracodawca,zawodowy,zakład,państwowy,umowa,pracowniczy,wynagrodzić,społeczny,zatrudnienie
Topic 2,służba,policja,straż,funkcjonariusz,celny,wewnętrzny,bezpieczeństwo,ochrona,administracja,graniczny
Topic 3,usta,arta,przepis,zapis,proponować,punkt,zakres,prawny,zasada,dzień
Topic 4,podatkowy,podatek,przepis,przedsiębiorca,publiczny,działalność,gospodarczy,skarbowy,podmiot,podatnik
Topic 5,dziecko,rodzina,kobieta,pomoc,rodzinny,społeczny,rodzic,problem,osoba,życie
Topic 6,rolnik,rolnictwo,rolny,gospodarstwo,produkcja,rolniczy,rynek,wieś,agencja,producent
Topic 7,ruch,drogowy,pojazd,samochód,kierowca,droga,wypadek,bezpieczeństwo,transport,prezydent
Topic 8,droga,inwestycja,krajowy,autostrada,budowy,infrastruktura,budowa,realizacja,program,środek
Topic 9,karny,kara,kodeks,przestępstwo,przepis,postępowanie,wolność,osoba,sprawca,pozbawić


In [80]:
bow_30_50_50 = funcs.print_topics(lda_model.print_topics(-1))

## LDA oparte o TF-IDF

In [66]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=5, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.004*"złoty" + 0.004*"budżet" + 0.003*"podatek" + 0.003*"mld" + 0.003*"rząd" + 0.003*"bank" + 0.003*"wzrost" + 0.002*"budżetowy" + 0.002*"pieniądz" + 0.002*"wydatek"
Topic: 1 Word: 0.005*"rolnik" + 0.004*"rolnictwo" + 0.004*"rolny" + 0.004*"europejski" + 0.003*"unia" + 0.003*"poprawka" + 0.002*"rynek" + 0.002*"rozwój" + 0.002*"podatek" + 0.002*"gospodarstwo"
Topic: 2 Word: 0.002*"rząd" + 0.002*"polska" + 0.002*"premier" + 0.002*"marszałek" + 0.002*"człowiek" + 0.002*"sejm" + 0.002*"prosić" + 0.002*"polityczny" + 0.001*"powiedzieć" + 0.001*"polak"
Topic: 3 Word: 0.003*"dziecko" + 0.003*"praca" + 0.003*"osoba" + 0.002*"zdrowie" + 0.002*"zmiana" + 0.002*"poprawka" + 0.002*"służba" + 0.002*"sąd" + 0.002*"świadczenie" + 0.002*"rodzina"
Topic: 4 Word: 0.011*"poprawka" + 0.006*"senat" + 0.003*"klub" + 0.003*"zmiana" + 0.003*"przyjąć" + 0.002*"głosować" + 0.002*"sprawozdanie" + 0.002*"nieruchomość" + 0.002*"zgłosić" + 0.002*"konwencja"


In [69]:
lda_model_tfidf.print_topics(0)

[(0,
  '0.004*"złoty" + 0.004*"budżet" + 0.003*"podatek" + 0.003*"mld" + 0.003*"rząd" + 0.003*"bank" + 0.003*"wzrost" + 0.002*"budżetowy" + 0.002*"pieniądz" + 0.002*"wydatek"'),
 (3,
  '0.003*"dziecko" + 0.003*"praca" + 0.003*"osoba" + 0.002*"zdrowie" + 0.002*"zmiana" + 0.002*"poprawka" + 0.002*"służba" + 0.002*"sąd" + 0.002*"świadczenie" + 0.002*"rodzina"'),
 (4,
  '0.011*"poprawka" + 0.006*"senat" + 0.003*"klub" + 0.003*"zmiana" + 0.003*"przyjąć" + 0.002*"głosować" + 0.002*"sprawozdanie" + 0.002*"nieruchomość" + 0.002*"zgłosić" + 0.002*"konwencja"'),
 (2,
  '0.002*"rząd" + 0.002*"polska" + 0.002*"premier" + 0.002*"marszałek" + 0.002*"człowiek" + 0.002*"sejm" + 0.002*"prosić" + 0.002*"polityczny" + 0.001*"powiedzieć" + 0.001*"polak"'),
 (1,
  '0.005*"rolnik" + 0.004*"rolnictwo" + 0.004*"rolny" + 0.004*"europejski" + 0.003*"unia" + 0.003*"poprawka" + 0.002*"rynek" + 0.002*"rozwój" + 0.002*"podatek" + 0.002*"gospodarstwo"')]

In [100]:
funcs.print_topics(lda_model_tfidf.print_topics(-1))

,Topic 0,Topic 1,Topic 2,Topic 3,Topic 4
0,złoty,rolnik,rząd,dziecko,poprawka
1,budżet,rolnictwo,polska,praca,senat
2,podatek,rolny,premier,osoba,klub
3,mld,europejski,marszałek,zdrowie,zmiana
4,rząd,unia,człowiek,zmiana,przyjąć
5,bank,poprawka,sejm,poprawka,głosować
6,wzrost,rynek,prosić,służba,sprawozdanie
7,budżetowy,rozwój,polityczny,sąd,nieruchomość
8,pieniądz,podatek,powiedzieć,świadczenie,zgłosić
9,wydatek,gospodarstwo,polak,rodzina,konwencja


In [85]:
lda_model_tfidf.print_topics(-1)[0]

(0,
 '0.004*"złoty" + 0.004*"budżet" + 0.003*"podatek" + 0.003*"mld" + 0.003*"rząd" + 0.003*"bank" + 0.003*"wzrost" + 0.002*"budżetowy" + 0.002*"pieniądz" + 0.002*"wydatek"')

In [89]:
pd.Series(a[0][1].split(' + ')).apply(lambda x : re.sub(r'[\d\.\*\"\']', '', x))

0        poprawka
1           senat
2            klub
3          zmiana
4         przyjąć
5        głosować
6    sprawozdanie
7    nieruchomość
8         zgłosić
9       konwencja
Name: Topic 4, dtype: object

In [91]:
topic_df = pd.DataFrame()
for topic in lda_model_tfidf.print_topics(-1):
    topic_df['Topic ' + str(topic[0])] = pd.Series(topic[1].split(' + ')).apply(lambda x : re.sub(r'[\d\.\*\"\']', '', x))

In [93]:
topic_df

,Topic 0,Topic 1,Topic 2,Topic 3,Topic 4
0,złoty,rolnik,rząd,dziecko,poprawka
1,budżet,rolnictwo,polska,praca,senat
2,podatek,rolny,premier,osoba,klub
3,mld,europejski,marszałek,zdrowie,zmiana
4,rząd,unia,człowiek,zmiana,przyjąć
5,bank,poprawka,sejm,poprawka,głosować
6,wzrost,rynek,prosić,służba,sprawozdanie
7,budżetowy,rozwój,polityczny,sąd,nieruchomość
8,pieniądz,podatek,powiedzieć,świadczenie,zgłosić
9,wydatek,gospodarstwo,polak,rodzina,konwencja
